In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Função 3

In [ ]:
from sympy import symbols

variaveis = list(symbols('x_1 x_2'))

print("Variáveis da função 3:")
display(variaveis)

In [ ]:
x_1, x_2 = variaveis

f_x = (x_1**2 + x_2 - 11)**2 + (x_1 + x_2**2 - 7)**2

print("Função 3:")
display(f_x)

In [ ]:
from algoritmos import calcula_gradiente

grad_f_x = calcula_gradiente(f_x, variaveis)

print("Vetor gradiente da função 3:")
display(grad_f_x)

In [ ]:
from algoritmos import calcula_hessiana

hess_f_x = calcula_hessiana(f_x, variaveis)

print("Matriz Hessiana da função 3:")
display(hess_f_x)

In [ ]:
# TODO: Estudo da função: pontos críticos, convexidade, existência de ponto(s) ótimo(s), além de plotar sua função, etc.

In [ ]:
# TODO: Achar os melhores valores de gama e eta

In [ ]:
import numpy as np
import pandas as pd

colunas = ['Ponto inicial', '# de iteracoes', '# de cham. de armijo', 'Ponto otimo', 'Valor otimo', 'Erro de aproximação']

pontos = [
  np.array([2.9, 2.1]), 
  np.array([-3.1, -1.9]), 
  # np.array([30, 20]), 
  np.array([4, -2]), 
  np.array([-3, 1]), 
  np.array([2.5, 1.5]), 
  np.array([-3.5, 2.5]), 
  # np.array([0, 0]), 
  np.array([5, 5]), 
  np.array([3.9, -0.9])
  ]

eta = 0.02
gama = 0.08

df_base = pd.DataFrame(columns=colunas)

print("Gama:", gama)
print("Eta:", eta)

print("Pontos a serem testados:")
display(pontos)

### Gradiente

In [ ]:
from algoritmos import metodo_do_gradiente
from utils import substitui_variaveis_funcao, substitui_variaveis_gradiente

df_gradiente = df_base.copy()

for ponto in pontos:
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto, gama, eta)
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_gradiente = pd.concat([df_gradiente, pd.DataFrame([[ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

df_gradiente

### Newton

In [ ]:
from algoritmos import metodo_de_newton

df_newton = df_base.copy()

for ponto in pontos:
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_newton(f_x, grad_f_x, hess_f_x, variaveis, ponto, gama, eta)
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_newton = pd.concat([df_newton, pd.DataFrame([[ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

display(df_newton)

### Quase-Newton

In [ ]:
from algoritmos import metodo_de_quase_newton

df_quase_newton = df_base.copy()

for metodo in ['bfgs', 'dfp']:
    for ponto in pontos:
        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_quase_newton(f_x, len(variaveis), grad_f_x, variaveis, ponto, gama, eta, metodo=metodo)
        valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
        erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
        df_quase_newton = pd.concat([df_quase_newton, pd.DataFrame([[ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao]], columns=colunas)])

display(df_quase_newton)

In [ ]:
np.nan_to_num(np.array([np.nan, -np.inf]))